# Experimenting with Package Functions

IN PROGRESS

## (1) Data generation with MOSFIT

In [ ]:
from survey_agnostic_sn_vae.data_generation.mosfit2 import *
from survey_agnostic_sn_vae.data_generation.objects import *
import os

OUTPUT_PATH = os.getcwd()

os.makedirs(OUTPUT_PATH+"/transients", exist_ok=True)

print(OUTPUT_PATH)
#for f in glob.glob(OUTPUT_PATH+"/transients/*"):
#    os.remove(f)
    
lsst_cadence = {
    'u': 25.0,
    'g': 17.0,
    'r': 6.0,
    'i': 7.0,
    'z': 8.0,
    'y': 8.0
}

panstarrs_cadence = {b: 3.0 for b in ['g', 'r', 'i', 'z']}

lsst_survey = Survey(
    'LSST',
    ['u', 'g', 'r', 'i', 'z', 'y'],
    lsst_cadence
)
ztf_survey = Survey(
    'ZTF',
    ['g', 'r'],
    2.0 # band sampling coupled
)
panstarrs_survey = Survey(
    'PanSTARRS',
    ['g', 'r', 'i', 'z'],
    panstarrs_cadence
)

generate_LCs_from_model(
    'slsn',
    [lsst_survey, ztf_survey, panstarrs_survey],
    10000,
    output_path=OUTPUT_PATH
);

In [ ]:
from survey_agnostic_sn_vae.data_generation.objects import *

OUTPUT_PATH = os.getcwd()

# Now, load Transient object
all_transient_fns = glob.glob(OUTPUT_PATH+"/transients/*")
test_transient = Transient.load(
    all_transient_fns[0],
)
print(test_transient.lightcurves) # should be 2
for lc in test_transient.lightcurves:
    print(lc.bands)
    print(lc.survey.bands)
    print(lc.survey.limiting_magnitude)

In [ ]:
import os
from survey_agnostic_sn_vae.preprocessing import (
    generate_superraenn_lc_file
)

OUTPUT_PATH = os.getcwd()

transient_dir = os.path.join(
    OUTPUT_PATH,
    "transients"
)
save_dir = os.path.join(
    OUTPUT_PATH,
    "superraenn"
)

os.makedirs(save_dir, exist_ok=True)

generate_superraenn_lc_file(
    transient_dir,
    save_dir
)

0
50
100
150
200
250
SKIPPED
300


In [ ]:
# overlay multiple LCs for same object 
from survey_agnostic_sn_vae.plotting import plot_lightcurves
from survey_agnostic_sn_vae.data_generation.objects import *

import os

OUTPUT_PATH = os.getcwd()

# Now, load Transient obje dct
all_transient_fns = glob.glob(OUTPUT_PATH+"/transients/*")
test_transient = Transient.load(
    all_transient_fns[1],
)

plot_lightcurves(
    test_transient,
    os.path.join(OUTPUT_PATH, "sample_transient.pdf")
)

## Test RAENN encodings

 (1) First try just VAE encodings without similarity loss

In [ ]:
from survey_agnostic_sn_vae.preprocessing import prep_input
from survey_agnostic_sn_vae.raenn import *
from keras.models import load_model
from survey_agnostic_sn_vae.custom_nn_layers.kl_loss import SamplingLayer
from survey_agnostic_sn_vae.custom_nn_layers.recon_loss import ReconstructionLoss
from keras.optimizers import Adam
import keras
import pretty_errors

OUTDIR = 'superraenn'
LCFILE = 'superraenn/lcs_ALL.npz'

NEURON_N_DEFAULT = 100
ENCODING_N_DEFAULT = 10
N_EPOCH_DEFAULT = 100
BATCH_SIZE = 128

sequence, outseq, ids, maxlen, nfilts = prep_input(LCFILE, save=True, outdir=OUTDIR)
model, callbacks_list, input_1, encoded = make_model(
    NEURON_N_DEFAULT, ENCODING_N_DEFAULT, int(maxlen), nfilts, N_EPOCH_DEFAULT, BATCH_SIZE
)
"""
model = load_model(
    OUTDIR + "/models/model.h5",
    custom_objects = {
        'SamplingLayer': SamplingLayer,
        'ReconstructionLoss': ReconstructionLoss
    },
    compile=False
)
"""
new_optimizer = Adam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999)
rl = ReconstructionLoss(6)
model.compile(optimizer=new_optimizer, loss=rl)
"""
# try adding a new layer!
model, callbacks_list, input_1, encoded = add_encoded_nodes(model, N_EPOCH_DEFAULT, n=1)
ENCODING_N_DEFAULT += 1
"""
model.summary()
for l in model.layers:
    print(l)

#annealing = AnnealingCallback(model.layers[4].beta,"cyclical",N_EPOCH_DEFAULT)    
#callbacks_list = [annealing,]
callbacks_list = []
model = fit_model(model, callbacks_list, sequence, outseq, N_EPOCH_DEFAULT, BATCH_SIZE)
encoder = get_encoder(model, input_1, encoded)

if OUTDIR[-1] != '/':
    OUTDIR += '/'
    
save_model(model, ENCODING_N_DEFAULT, NEURON_N_DEFAULT, outdir=OUTDIR)

save_encodings( 
    model, encoder, sequence, ids, LCFILE,
    ENCODING_N_DEFAULT, NEURON_N_DEFAULT, len(ids), maxlen,
    OUTDIR
)

Platform 'METAL' is experimental and not all JAX functionality may be correctly supported!


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

METAL


2024-04-13 22:55:58.468961: W pjrt_plugin/src/mps_client.cc:563] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!


making model


Model: "jax_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None, 19)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, None, 100) │     62,300 │ input_layer[0][0] │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 100)       │          0 │ sequential[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 1098, 100) │          0 │ get_item[0][0]    │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 1098, 100) │          0 │ repeat_vector[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 1098, 1)   │     10,201 │ get_item_1[0][0]  │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 1098, 3)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 183, 6) │          0 │ sequential_1[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 72,501 (283.21 KB)

 Trainable params: 72,501 (283.21 KB)

 Non-trainable params: 0 (0.00 B)

<InputLayer name=input_layer, built=True>
<Sequential name=sequential, built=True>
<RepeatVector name=repeat_vector, built=True>
<Sequential name=sequential_1, built=True>
<InputLayer name=input_layer_1, built=True>
<InputLayer name=input_layer_2, built=True>
<Reshape name=reshape, built=True>
Epoch 1/100
STARTING TRAINING


In [ ]:
from survey_agnostic_sn_vae.preprocessing import prep_input
from survey_agnostic_sn_vae.raenn import *
from survey_agnostic_sn_vae.plotting import *

from tensorflow.keras.models import load_model
from survey_agnostic_sn_vae.custom_nn_layers.kl_loss import SamplingLayer
from survey_agnostic_sn_vae.custom_nn_layers.recon_loss import ReconstructionLoss
from tensorflow.keras.optimizers.legacy import Adam
import tensorflow as tf

OUTDIR = 'superraenn'
LCFILE = 'superraenn/lcs.npz'

# overlay decodings
fig, ax = plt.subplots()

"""
model = load_model(
    OUTDIR + "/models/model.h5",
    custom_objects = {
        'SamplingLayer': SamplingLayer,
        'ReconstructionLoss': ReconstructionLoss
    },
    compile=False
)
"""
sequence, outseq, ids, maxlen, nfilts = prep_input(LCFILE, save=True, outdir=OUTDIR)

decodings = get_decodings(
    model, sequence[:10], outseq[:10]
)

bands = 'ugrizy'

for i in range(10):
    seq = sequence[i]
    d = np.reshape(decodings[i], (seq.shape[0], -1))
    for b_idx in range(6):
        c_i = BAND_COLORS[bands[b_idx]]
        plt.errorbar(seq[:,0], seq[:,b_idx+1], yerr=seq[:,7+b_idx], fmt='none', ecolor=c_i)
        plt.scatter(seq[:,0], seq[:,b_idx+1], marker='o', color=c_i)
        plt.plot(seq[:,0], d[:,b_idx], c=c_i)
        
    plt.show()
